# Build the basin example base model

## Notebook Setup

In [ ]:
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib as pl
from shapely.geometry import Polygon, LineString
import flopy
from flopy.discretization import StructuredGrid

In [ ]:
# import containerized functionality from defaults.py
from defaults import *

In [ ]:
fine_topo = flopy.utils.Raster.load("./grid_data/fine_topo.asc")

### Parallel settings

1. Set `path_to_mf6` to the path of parallel MODFLOW 6 (`path\to\mf6`) if it is not in your `PATH` otherwise set to `None`.
2. Set `binary_input` to `True` to write distributed array and stress period data to binary files.

In [ ]:
path_to_mf6 = pl.Path.home() / ".local/bin/mf6"
binary_input = True

# Basin Structured Example

Structured grid parameters

In [ ]:
dx = dy = 500.0  # 166.666666667
nrow = int(Ly / dy) + 1
ncol = int(Lx / dx) + 1

Read in boundary data

In [ ]:
boundary_polygon = string2geom(geometry["boundary"])
print("Length boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)

stream_segs = (
    geometry["streamseg1"],
    geometry["streamseg2"],
    geometry["streamseg3"],
    geometry["streamseg4"],
)
sgs = [string2geom(sg) for sg in stream_segs]


fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
ax.set_aspect("equal")

riv_colors = ("blue", "cyan", "green", "orange", "red")

ax.plot(bp[:, 0], bp[:, 1], "ro-")
for idx, sg in enumerate(sgs):
    print("Length segment: ", len(sg))
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], color=riv_colors[idx], lw=0.75, marker="o")

Create a structured grid

In [ ]:
working_grid = StructuredGrid(
    nlay=1,
    delr=np.full(ncol, dx),
    delc=np.full(nrow, dy),
    xoff=0.0,
    yoff=0.0,
    top=np.full((nrow, ncol), 1000.0),
    botm=np.full((1, nrow, ncol), -100.0),
)

set_structured_idomain(working_grid, boundary_polygon)
print(Lx, Ly, nrow, ncol)

Sample topography

In [ ]:
top_wg = fine_topo.resample_to_grid(
    working_grid,
    band=fine_topo.bands[0],
    method="linear",
    extrapolate_edges=True,
)

Intersect river segments with grid

In [ ]:
ixs, cellids, lengths = intersect_segments(working_grid, sgs)

Plot the river intersection

In [ ]:
intersection_rg = np.zeros(working_grid.shape[1:])
for loc in cellids:
    intersection_rg[loc] = 1

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
pmv = flopy.plot.PlotMapView(modelgrid=working_grid)
ax.set_aspect("equal")
pmv.plot_array(top_wg)
pmv.plot_array(
    intersection_rg,
    masked_values=[
        0,
    ],
    alpha=0.2,
    cmap="Reds_r",
)
pmv.plot_inactive()
ax.plot(bp[:, 0], bp[:, 1], "r-")
for sg in sgs:
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], "b-")

Set the idomain value to 2 where the river intersects the grid

In [ ]:
river_locations = working_grid.idomain[0].copy()
index = tuple(np.array(list(zip(*cellids))))
river_locations[index] = 2
working_grid.idomain = river_locations.reshape(1, nrow, ncol)

### Build the base model

Set the simulation workspace

In [ ]:
sim_ws = get_base_workspace()
shutil.rmtree(sim_ws, ignore_errors=True)
sim_ws

Define the number of layers and layer 1 thickness

In [ ]:
nlay = 5
dv0 = 5.0

Create the drain data for the river segments

In [ ]:
leakance = 1.0 / (0.5 * dv0)  # kv / b
drn_data = build_drain_data(
    working_grid,
    cellids,
    lengths,
    leakance,
    top_wg,
)
drn_data[:10]

Create the groundwater discharge drain data

In [ ]:
gw_discharge_data = build_groundwater_discharge_data(
    working_grid,
    leakance,
    top_wg,
)
gw_discharge_data[:10]

Use binary files for top, recharge_data, drn_data, and gw_discharge_data if `binary_input` is `True`. Otherwise write external ascii files.

In [ ]:
if binary_input:
    external_ext = ".bin"
else:
    external_ext = ".txt"

In [ ]:
lse_data = {
    "filename": f"lse{external_ext}",
    "binary": binary_input,
    "data": top_wg,
}

recharge_data = {
            0: {
                "filename": f"recharge{external_ext}",
                "binary": binary_input,
                "data": 0.000001,
            },
        }

drn_data = {
            0: {
                "filename": f"drn_riv{external_ext}",
                "binary": binary_input,
                "data": drn_data,
            },
        }

gw_discharge_data = {
            0: {
                "filename": f"drn_gwd{external_ext}",
                "binary": binary_input,
                "data": gw_discharge_data,
            },
        }

Create the top and bottom arrays. Top array is not used by the model.

In [ ]:
topc = np.zeros((nlay, nrow, ncol), dtype=float)
botm = np.zeros((nlay, nrow, ncol), dtype=float)
dv = dv0
topc[0] = top_wg.copy()
botm[0] = topc[0] - dv
for idx in range(1, nlay):
    dv *= 1.5
    topc[idx] = botm[idx - 1]
    botm[idx] = topc[idx] - dv

Print the cell thicknesses

In [ ]:
for k in range(nlay):
    print((topc[k] - botm[k]).mean())

Create the idomain and starting heads for the model

In [ ]:
idomain = np.array(
    [working_grid.idomain[0, :, :].copy() for k in range(nlay)], dtype=int
)
strt = np.array([top_wg.copy() for k in range(nlay)], dtype=float)

### Build the model files using FloPy

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name="basin",
    sim_ws=sim_ws,
    exe_name="mf6",
    memory_print_option="summary",
)

tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(
    sim,
    complexity="simple",
    print_option="SUMMARY",
    linear_acceleration="bicgstab",
    outer_maximum=1000,
    inner_maximum=100,
    outer_dvclose=1e-5,
    inner_dvclose=1e-6,
    preconditioner_levels=2,
    relaxation_factor=0.0,
)
gwf = flopy.mf6.ModflowGwf(
    sim,
    save_flows=True,
    newtonoptions="NEWTON UNDER_RELAXATION",
)

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=dx,
    delc=dy,
    idomain=idomain,
    top=lse_data,
    botm=botm,
    xorigin=0.0,
    yorigin=0.0,
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=1,
    k=1.0,
)
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=recharge_data,
)
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_data,
    pname="river",
    filename="drn_riv.drn",
)
drn_gwd = flopy.mf6.ModflowGwfdrn(
    gwf,
    auxiliary=["depth"],
    auxdepthname="depth",
    stress_period_data=gw_discharge_data,
    pname="gwd",
    filename="drn_gwd.drn",
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)

Count the number of active cells

In [ ]:
total_cells, active_cells = get_simulation_cell_count(sim)
total_cells, active_cells

### Write the model files

In [ ]:
sim.write_simulation()

In [ ]:
write_petscdb(sim_ws, use_gamg=False)

### Run the model

In [ ]:
if local_simulation():
    if path_to_mf6 is not None:
        sim.exe_name = path_to_mf6
    sim.run_simulation(processors=1)